# Data Cleaning

` Data cleaning is a time consuming and unenjoyable task, yet it's a very important one. Keep in mind, "garbage in, garbage out".`

#### Feeding dirty data into a model will give us results that are meaningless.

### Objective:

1. Getting the data 
2. Cleaning the data 
3. Organizing the data - organize the cleaned data into a way that is easy to input into other algorithms

### Output :
#### cleaned and organized data in two standard text formats:

1. Corpus - a collection of text
2. Document-Term Matrix - word counts in matrix format

## Problem Statement

Look at transcripts of various comedians and note their similarities and differences and find if the stand up comedian of your choice has comedy style different than other comedian.


## Getting The Data

You can get the transcripts of some comedian from [Scraps From The Loft](http://scrapsfromtheloft.com). 

You can take help of IMDB and select only 10 or 20 comedian having highest rating.






### For example:

In [10]:
import requests
from bs4 import BeautifulSoup
import pickle
import nltk
from nltk.corpus import stopwords

# Scrapes transcript data from scrapsfromtheloft.com
def url_to_transcript(url):
    '''Returns transcript data specifically from scrapsfromtheloft.com.'''
    r= requests.get(url)
    htmlContent = r.content
    soup = BeautifulSoup(htmlContent, "html.parser")
    text = [p.text for p in soup.find_all('p',{'style': 'text-align: justify;'})]
    print(url)
    return text


# URLs of transcripts in scope
urls = ['http://scrapsfromtheloft.com/comedy/gabriel-iglesias-stadium-fluffy-transcript/',
        'http://scrapsfromtheloft.com/comedy/patton-oswalt-we-all-scream-transcript/',
        'http://scrapsfromtheloft.com/comedy/bill-burr-live-at-red-rocks-transcript/',
        'http://scrapsfromtheloft.com/comedy/kathleen-madigan-bothering-jesus-transcript/',
        'http://scrapsfromtheloft.com/comedy/vir-das-losing-it-transcript/',
        'http://scrapsfromtheloft.com/comedy/jim-gaffigan-quality-time-transcript/',
        'http://scrapsfromtheloft.com/comedy/fortune-feimster-good-fortune-transcript/',
        'http://scrapsfromtheloft.com/comedy/brian-regan-standing-up-2007-full-transcript/',
        'http://scrapsfromtheloft.com/2017/04/11/dave-chappelle-age-spin-2017-full-transcript/'
        ]

# Comedian names
comedians = ['gabriel', 'oswalt','bill', 'kathleen', 'vir','jim','fortune','brian','dave']

In [11]:
# # Actually request transcripts (takes a few minutes to run)
transcripts = [url_to_transcript(u) for u in urls]

http://scrapsfromtheloft.com/comedy/gabriel-iglesias-stadium-fluffy-transcript/
http://scrapsfromtheloft.com/comedy/patton-oswalt-we-all-scream-transcript/
http://scrapsfromtheloft.com/comedy/bill-burr-live-at-red-rocks-transcript/
http://scrapsfromtheloft.com/comedy/kathleen-madigan-bothering-jesus-transcript/
http://scrapsfromtheloft.com/comedy/vir-das-losing-it-transcript/
http://scrapsfromtheloft.com/comedy/jim-gaffigan-quality-time-transcript/
http://scrapsfromtheloft.com/comedy/fortune-feimster-good-fortune-transcript/
http://scrapsfromtheloft.com/comedy/brian-regan-standing-up-2007-full-transcript/
http://scrapsfromtheloft.com/2017/04/11/dave-chappelle-age-spin-2017-full-transcript/


In [14]:
# Pickle files for later use
# Make a new directory to hold the text files
!mkdir transcripts

for i, c in enumerate(comedians):
    with open("transcripts/" + c + ".txt", "wb") as file:
        pickle.dump(transcripts[i], file)

In [15]:
# Load pickled files
data = {}
for i, c in enumerate(comedians):
    with open("transcripts/" + c + ".txt", "rb") as file:
        data[c] = pickle.load(file)

In [16]:
# Double check to make sure data has been loaded properly
data.keys()

dict_keys(['gabriel', 'oswalt', 'bill', 'kathleen', 'vir', 'jim', 'fortune', 'brian', 'dave'])

In [17]:
# More checks
data['oswalt'][:20]

['[“Hard to Kill” by Bleached plays]',
 '[audience cheering and applauding]',
 'Hello! Denver!',
 '[cheering and applause continues]',
 '[Patton laughs] Oh my God! Hello. Thank you. Thank you. Thank you.',
 '[cheering continues]',
 'Thank you. Oh my God! Yes! Thank you all so much for coming out tonight. Um… Hey, I broke my foot. Yeah. That’s how I started my year. I broke my foot. Uh… It was… It took two… It happened in two parts. Uh, the second part was I slipped off a curb and I landed wrong. That was the second part. The first and most important part was I turned 53. That is the crucial part.',
 '[audience laughs]',
 'I… You turn… Once you get past 50, everything’s fatal. I… When I was in my twenties, I would walk into propellers and put some Bactine on it. I was fine. And now if a pine cone falls near me, my spine implodes. I don’t know what… Like that. [audience laughs] Broke my foot, and then the day after… I’m gonna drop a name. All right. Uh, the day after I broke my foot, Gle

## Cleaning The Data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate.
### Assignment:
1. Perform the following data cleaning on transcripts:
i) Make text all lower case
ii) Remove punctuation
iii) Remove numerical values
iv) Remove common non-sensical text (/n)
v) Tokenize text
vi) Remove stop words

In [18]:
# Let's take a look at our data again
next(iter(data.keys()))

'gabriel'

In [19]:
# Notice that our dictionary is currently in key: comedian, value: list of text format
next(iter(data.values()))

['[man] Can you please state your name? Martin Moreno. But you might know me as… Martinnnnn! I’ve been touring with Gabriel Iglesias for 20-plus years. Martinnnnn! And, yeah, he’s been screaming my name for 20-plus years. Hurry up, Martinnnnn! Dude, that’s better than most marriages. That’s a win. It has been an incredible journey. We’ve gone from garages, clubs, living rooms, to theaters, arenas around the world, and now a stadium. They say comedy is subjective, but when you’re selling out stadiums, that’s no longer subjective. So, what can people expect? One of the biggest shows Los Angeles has ever seen. And whatever you do, make sure you stick around till the end. You want to see how this thing ends. So, what else is left to say? Without further ado, live from Dodger Stadium, Gabriel Iglesias!',
 '[theme playing from 2001: A Space Odyssey]',
 '[ignition cranking]',
 '[engine not starting]',
 '[ignition cranking]',
 '[engine starts]',
 '♪ California love ♪ ♪ California ♪ ♪ Knows how

In [20]:
# We are going to change this to key: comedian, value: string format
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

In [21]:
# Combine it!
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [22]:
# We can either keep it in dictionary format or put it into a pandas dataframe
import pandas as pd
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df

,transcript
bill,"[audience cheering, whistling] [emcee] Ladies and gentlemen, Bill Burr! [cheering and applause] All right, thank you! Thank you very much. Thank y..."
brian,"♪ [Rock] [crowd cheering] [man] let’s give a big, warm welcome to Mr. Brian Regan! [Crowd roars] all right. Thank you! Thank you. Thanks. [Cheerin..."
dave,"This is Dave. He tells dirty jokes for a living. That stare is where most of his hard work happens. It signifies a profound train of thought, the ..."
fortune,[upbeat music plays] [audience cheering] [announcer] Please welcome Fortune Feimster! ♪ I’m a powerful woman ♪ ♪ Always get what I want ♪ ♪ So don...
gabriel,[man] Can you please state your name? Martin Moreno. But you might know me as… Martinnnnn! I’ve been touring with Gabriel Iglesias for 20-plus yea...
jim,"(audience cheering, applauding) Thank you! Thank you! Oh my God. Thank you. Thank you for coming out. (audience cheering) It’s good to be here. Th..."
kathleen,[chattering] [man] Whoo! [chattering] [woman] ♪ Kathleen ♪ ♪ Madigan ♪ ♪ Kathleen ♪ ♪ Madigan ♪ ♪ She’s coming on the stage now ♪ [audience laughs...
oswalt,[“Hard to Kill” by Bleached plays] [audience cheering and applauding] Hello! Denver! [cheering and applause continues] [Patton laughs] Oh my God! ...
vir,"I lost 80% of my mind. It’s very freeing. You should see the look on your faces right now, by the way. Oh! Good evening, San Francisco. Are you gu..."


In [23]:
# Let's take a look at the transcript for Ali Wong
data_df.transcript.loc['vir']

'I lost 80% of my mind. It’s very freeing. You should see the look on your faces right now, by the way. Oh! Good evening, San Francisco. Are you guys excited, yeah? All right. Well, my name is Vir Das. We’re gonna have such a good time tonight. I’m so excited. It’s gonna be delightful. Oh, this is how I talk now. I just thought it was time to really embrace my roots, you know, and to make my comedy more authentically Indian. And really, what could be more Indian than a fake American accent? I don’t think you understand. I have an opportunity to make history tonight, guys. I can. I can be the first ever Indian who comes to California… and then leaves. That’s never happened before. Because you guys are sticking around. Until they kick you out. Which, going by the news, is about three weeks from now. Now, I used to work in America in the year 2002. It didn’t go very well, so I left for browner pastures. And then, honestly, I didn’t think about you guys for 15 years. Because, honestly, the

In [24]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [25]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean

,transcript
bill,ladies and gentlemen bill burr all right thank you thank you very much thank you thank you thank you how are ya how’s it goin’ all right you gu...
brian,♪ let’s give a big warm welcome to mr brian regan all right thank you thank you thanks thank you very much appreciate that all right thank...
dave,this is dave he tells dirty jokes for a living that stare is where most of his hard work happens it signifies a profound train of thought the alch...
fortune,please welcome fortune feimster ♪ i’m a powerful woman ♪ ♪ always get what i want ♪ ♪ so don’t you get in my way now that’s not what i want ♪ ♪...
gabriel,can you please state your name martin moreno but you might know me as… martinnnnn i’ve been touring with gabriel iglesias for years martinnnnn a...
jim,audience cheering applauding thank you thank you oh my god thank you thank you for coming out audience cheering it’s good to be here this is what ...
kathleen,whoo ♪ kathleen ♪ ♪ madigan ♪ ♪ kathleen ♪ ♪ madigan ♪ ♪ she’s coming on the stage now ♪ ♪ she’s gonna be here shortly ♪ ♪ she even put some ...
oswalt,hello denver oh my god hello thank you thank you thank you thank you oh my god yes thank you all so much for coming out tonight um… hey i bro...
vir,i lost of my mind it’s very freeing you should see the look on your faces right now by the way oh good evening san francisco are you guys excited...


In [26]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [27]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.transcript.apply(round2))
data_clean

,transcript
bill,ladies and gentlemen bill burr all right thank you thank you very much thank you thank you thank you how are ya hows it goin all right you guys...
brian,♪ lets give a big warm welcome to mr brian regan all right thank you thank you thanks thank you very much appreciate that all right thank ...
dave,this is dave he tells dirty jokes for a living that stare is where most of his hard work happens it signifies a profound train of thought the alch...
fortune,please welcome fortune feimster ♪ im a powerful woman ♪ ♪ always get what i want ♪ ♪ so dont you get in my way now thats not what i want ♪ ♪ ca...
gabriel,can you please state your name martin moreno but you might know me as martinnnnn ive been touring with gabriel iglesias for years martinnnnn and...
jim,audience cheering applauding thank you thank you oh my god thank you thank you for coming out audience cheering its good to be here this is what i...
kathleen,whoo ♪ kathleen ♪ ♪ madigan ♪ ♪ kathleen ♪ ♪ madigan ♪ ♪ shes coming on the stage now ♪ ♪ shes gonna be here shortly ♪ ♪ she even put some pa...
oswalt,hello denver oh my god hello thank you thank you thank you thank you oh my god yes thank you all so much for coming out tonight um hey i brok...
vir,i lost of my mind its very freeing you should see the look on your faces right now by the way oh good evening san francisco are you guys excited ...


## Organizing The Data

### Assignment:
1. Organized data in two standard text formats:
   a) Corpus - corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.
   b) Document-Term Matrix - word counts in matrix format

### Corpus: Example

A corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.

In [28]:
# Let's take a look at our dataframe
data_df

,transcript
bill,"[audience cheering, whistling] [emcee] Ladies and gentlemen, Bill Burr! [cheering and applause] All right, thank you! Thank you very much. Thank y..."
brian,"♪ [Rock] [crowd cheering] [man] let’s give a big, warm welcome to Mr. Brian Regan! [Crowd roars] all right. Thank you! Thank you. Thanks. [Cheerin..."
dave,"This is Dave. He tells dirty jokes for a living. That stare is where most of his hard work happens. It signifies a profound train of thought, the ..."
fortune,[upbeat music plays] [audience cheering] [announcer] Please welcome Fortune Feimster! ♪ I’m a powerful woman ♪ ♪ Always get what I want ♪ ♪ So don...
gabriel,[man] Can you please state your name? Martin Moreno. But you might know me as… Martinnnnn! I’ve been touring with Gabriel Iglesias for 20-plus yea...
jim,"(audience cheering, applauding) Thank you! Thank you! Oh my God. Thank you. Thank you for coming out. (audience cheering) It’s good to be here. Th..."
kathleen,[chattering] [man] Whoo! [chattering] [woman] ♪ Kathleen ♪ ♪ Madigan ♪ ♪ Kathleen ♪ ♪ Madigan ♪ ♪ She’s coming on the stage now ♪ [audience laughs...
oswalt,[“Hard to Kill” by Bleached plays] [audience cheering and applauding] Hello! Denver! [cheering and applause continues] [Patton laughs] Oh my God! ...
vir,"I lost 80% of my mind. It’s very freeing. You should see the look on your faces right now, by the way. Oh! Good evening, San Francisco. Are you gu..."


In [29]:
# Let's add the comedians' full names as well
full_names = ['gabriel', 'oswalt','bill', 'kathleen', 'vir','jim','fortune','brian']

data_df['full_name'] = comedians
data_df

,transcript,full_name
bill,"[audience cheering, whistling] [emcee] Ladies and gentlemen, Bill Burr! [cheering and applause] All right, thank you! Thank you very much. Thank y...",gabriel
brian,"♪ [Rock] [crowd cheering] [man] let’s give a big, warm welcome to Mr. Brian Regan! [Crowd roars] all right. Thank you! Thank you. Thanks. [Cheerin...",oswalt
dave,"This is Dave. He tells dirty jokes for a living. That stare is where most of his hard work happens. It signifies a profound train of thought, the ...",bill
fortune,[upbeat music plays] [audience cheering] [announcer] Please welcome Fortune Feimster! ♪ I’m a powerful woman ♪ ♪ Always get what I want ♪ ♪ So don...,kathleen
gabriel,[man] Can you please state your name? Martin Moreno. But you might know me as… Martinnnnn! I’ve been touring with Gabriel Iglesias for 20-plus yea...,vir
jim,"(audience cheering, applauding) Thank you! Thank you! Oh my God. Thank you. Thank you for coming out. (audience cheering) It’s good to be here. Th...",jim
kathleen,[chattering] [man] Whoo! [chattering] [woman] ♪ Kathleen ♪ ♪ Madigan ♪ ♪ Kathleen ♪ ♪ Madigan ♪ ♪ She’s coming on the stage now ♪ [audience laughs...,fortune
oswalt,[“Hard to Kill” by Bleached plays] [audience cheering and applauding] Hello! Denver! [cheering and applause continues] [Patton laughs] Oh my God! ...,brian
vir,"I lost 80% of my mind. It’s very freeing. You should see the look on your faces right now, by the way. Oh! Good evening, San Francisco. Are you gu...",dave


In [30]:
# Let's pickle it for later use
data_df.to_pickle("corpus.pkl")

### Document-Term Matrix: Example

For many of the techniques we'll be using in future assignment, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's ` CountVectorizer `, where every row will represent a different document and every column will represent a different word.

In addition, with ` CountVectorizer `, we can remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [31]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names_out())
data_dtm.index = data_clean.index
data_dtm

,aaah,aarp,abc,abdomen,abducted,abhorring,ability,abki,able,aboard,...,zhoosh,zip,zipper,zoned,zones,zoom,zoomed,álvarez,ándale,ñañaras
bill,0,1,0,0,0,0,1,0,1,0,...,0,0,0,1,0,2,0,0,0,0
brian,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
dave,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fortune,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,1,1,0,0,0
gabriel,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,4,0,3,1,1
jim,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
kathleen,0,0,0,0,1,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
oswalt,0,0,0,0,0,0,0,0,2,1,...,1,0,0,0,0,2,0,0,0,0
vir,0,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Let's pickle it for later use
data_dtm.to_pickle("dtm.pkl")

In [33]:
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))

## Additional Assignments:

1. Can you add an additional regular expression to the clean_text_round2 function to further clean the text?
2. Play around with CountVectorizer's parameters. What is ngram_range? What is min_df and max_df?